In [6]:
import numpy as np 
from scipy.integrate import odeint
import os, sys 
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate1 as sr1
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns
import sample as sm
import hr_full as hr
import hr_restrict as hrr
# warnings.filterwarnings('ignore')

In [ ]:
D, D_r = 3, 2048
Lyapunov_time, dt = 1./.91, 0.02
error_threshold = 0.05
beta = 1e-14
L0, L1 = 0.4, 3.5
percents = 50
n_repeats = 100
error_threshold = 0.05
L63_data_folder = '../data/L63-trajectories'
train = np.load('{}/validation.npy'.format(L63_data_folder))
test = np.load('{}/test.npy'.format(L63_data_folder))[:, :1000, :]
save_folder = f'../data/small-beta/{D_r}'
batch = sr1.BatchStrategy_SMLR(save_folder, D, D_r, n_repeats, beta, error_threshold, train, test, dt=0.02, Lyapunov_time=1./0.91,\
                 L0=0.4, L1=3.5, percents=51, row_selection='good_linear_extreme', train_option='random_20000',\
                 limits_W_in=[-0.1, 0.1], limits_W=[-1., 1.])
batch.run(save_data=True)

Generating parameters for 0.0% good rows ...


/Users/pman0581/Documents/GitHub/random-feature-map/modules/sample.py:599: RuntimeWarning: overflow encountered in add
  x += t*d
/Users/pman0581/Documents/GitHub/random-feature-map/modules/sample.py:560: RuntimeWarning: invalid value encountered in subtract
  self.x0 = x0 - np.dot(x0, d) * d
/Users/pman0581/Documents/GitHub/random-feature-map/modules/sample.py:599: RuntimeWarning: overflow encountered in add
  x += t*d
/Users/pman0581/Documents/GitHub/random-feature-map/modules/sample.py:560: RuntimeWarning: invalid value encountered in subtract
  self.x0 = x0 - np.dot(x0, d) * d
